# Export module for Kaggle 

This notebook is for gathering all the definitions (`def`s, `class`s, etc.) that will be needed in the submission kernel into a single `.py` file.

In [2]:
from fastai.vision import *

In [3]:
SRC = Path('../kgl_deepfake/')

In [10]:
fnames_src = get_files(SRC, extensions='.py')
fnames_src = [fn for fn in fnames_src if not fn.name.startswith('_')]

In [11]:
IMPORT_LINES = ('from', 'import')

In [12]:
imps, defs = [], []
for fname in sorted(fnames_src):
    ls = open(fname).readlines()[4:]
    ls = [l for l in ls if not l.startswith('# Cell')]
    ls = [l+'\n' if not l.endswith('\n') else l for l in ls]
    imps_ls = [l for l in ls if l.startswith(IMPORT_LINES)]
    defs_ls = [l for l in ls if not l.startswith(IMPORT_LINES)]
    imps.extend(imps_ls); defs.extend(defs_ls)

In [13]:
imps = sorted([l for l in set(imps) if not l.startswith('from .nb_')])
imps = [l for l in imps if not l.startswith('from nbdev')]

In [14]:
with open('kernel_module.py', mode='w', encoding='utf-8') as f:
    f.write(''.join(imps + defs))

In [15]:
! cat kernel_module.py

from IPython.display import HTML
from IPython.display import display, Video, HTML
from facenet_pytorch import MTCNN
from fastai.core import *
from fastai.vision import *
from tqdm import tqdm
import cv2
import pandas as pd
import time
import torch.nn as nn
import torch.nn.functional as F


def html_vid(fname, **kwargs): return display(Video(fname, **kwargs))

def html_vid(fname):
    "Return HTML for video."
    return f'''
    <video width="300" height="250" controls>
    <source src="{fname}" type="video/mp4">
    </video>
    '''

def html_titled_vid(fname, title):
    "Return HTML for titled video."
    return f'<div><p>{title}</p><br>{html_vid(fname)}</div>'

def html_vids(fnames, titles=None, ncols=3):
    "Return HTML for table of (titled) videos."
    n = len(fnames)
    if titles is None: titles = n * ['']
    assert len(titles) == n
    rs = []
    for i in range(0, n, ncols):
        fs, ts = fnames[i:i+ncols], titles[i:i+ncols]
        xs = (html_titled_vid(f, t) for f,t in